### The Number of Rich Customers

In [2]:
import pandas as pd
import pandasql as ps

data = [[6, 1, 549], [8, 1, 834], [4, 2, 394], [11, 3, 657], [13, 3, 257]]
Store = pd.DataFrame(data, columns=['bill_id', 'customer_id', 'amount']).astype({'bill_id':'int64', 'customer_id':'int64', 'amount':'int64'})
print(Store)

   bill_id  customer_id  amount
0        6            1     549
1        8            1     834
2        4            2     394
3       11            3     657
4       13            3     257


In [21]:
result_df = Store.groupby('customer_id').max('amount')
print(pd.DataFrame([result_df[result_df['amount']>500]['amount'].count()], columns=['rich_count']))

   rich_count
0           2


In [38]:
query = '''

select count(*) as rich_count
from (select max(amount) 
from Store
group by customer_id
having amount > 500)
'''

ps.sqldf(query,locals())

,rich_count
0,2


### Immediate Food Delivery I

In [81]:
data = [[1, 1, '2019-08-01', '2019-08-02'], [2, 5, '2019-08-02', '2019-08-02'], [3, 1, '2019-08-11', '2019-08-11'], [4, 3, '2019-08-24', '2019-08-26'], [5, 4, '2019-08-21', '2019-08-22'], [6, 2, '2019-08-11', '2019-08-13']]
Delivery = pd.DataFrame(data, columns=['delivery_id', 'customer_id', 'order_date', 'customer_pref_delivery_date']).astype({'delivery_id':'Int64', 'customer_id':'Int64', 'order_date':'datetime64[ns]', 'customer_pref_delivery_date':'datetime64[ns]'})
print(Delivery)

   delivery_id  customer_id order_date customer_pref_delivery_date
0            1            1 2019-08-01                  2019-08-02
1            2            5 2019-08-02                  2019-08-02
2            3            1 2019-08-11                  2019-08-11
3            4            3 2019-08-24                  2019-08-26
4            5            4 2019-08-21                  2019-08-22
5            6            2 2019-08-11                  2019-08-13


In [79]:
round(len(Delivery[Delivery['customer_pref_delivery_date'] == Delivery['order_date']])/len(Delivery)*100,2)

33.33

In [118]:
query = '''
with immediate_orders as(
    select count(*) as immediate_count
    from Delivery
    where customer_pref_delivery_date = order_date
),
total_orders as (
    select count(*) as total_count
    from Delivery
)

select round((cast(immediate_count as float) / cast(total_count as float) * 100),2) as immediate_percentage
from immediate_orders
cross join total_orders
'''

 
ps.sqldf(query,locals())

,immediate_percentage
0,33.33


### Count Salary Categories

In [174]:
data = [[3, 108939], [2, 12747], [8, 87709], [6, 91796]]
accounts = pd.DataFrame(data, columns=['account_id', 'income']).astype({'account_id':'Int64', 'income':'Int64'})
print(accounts)

   account_id  income
0           3  108939
1           2   12747
2           8   87709
3           6   91796


In [178]:
print(pd.DataFrame({
    'category' : ['Low Salary', 'Average Salary', 'High Salary'],
    'accounts_count' : [
        accounts[accounts.income < 20000].shape[0],
        accounts[(accounts.income >= 20000)&(accounts.income < 50000)].shape[0],
        accounts[accounts.income > 50000].shape[0]
    ]
}))

         category  accounts_count
0      Low Salary               1
1  Average Salary               0
2     High Salary               3


In [190]:
query = '''

select category, accounts_count
from(select "Low Salary" as category, sum(income < 20000) as accounts_count
from Accounts

union

select "Average Salary" as category, sum(income between 20000 and 50000) as accounts_count
from Accounts

union

select "High Salary" as category, sum(income >= 50000) as accounts_count
from Accounts)

order by case 
when category = 'Low Salary' then 1 
when category = 'Average Salary' then 2
else 3
end;
'''

print(ps.sqldf(query, locals()))

         category  accounts_count
0      Low Salary               1
1  Average Salary               0
2     High Salary               3
